<a href="https://colab.research.google.com/github/bysliu/DATA205/blob/main/data205_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import holidays



In [ ]:
#Datasets I want to use.

driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250302.csv', parse_dates=['Crash Date/Time'])
incidents_data = pd.read_csv('Crash_Reporting_-_Incidents_Data_20250302.csv',  parse_dates=['Crash Date/Time'])
nonmotor_data = pd.read_csv('Crash_Reporting_-_Non-Motorists_Data_20250302.csv',  parse_dates=['Crash Date/Time'])

driver_data_copy = driver_data.copy()
incidents_data_copy = incidents_data.copy()
nonmotor_data_copy = nonmotor_data.copy()

<ipython-input-2-8676bb2ded1a>:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250302.csv', parse_dates=['Crash Date/Time'])
<ipython-input-2-8676bb2ded1a>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250302.csv', parse_dates=['Crash Date/Time'])
<ipython-input-2-8676bb2ded1a>:4: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents_data = pd.read_csv('Crash_Reporting_-_Incidents_Data_20250302.csv',  parse_dates=['Crash Date/Time'])
<ipython-input-2-8676bb2ded1a>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and a

In [ ]:
#Merging Crash datasets

merge_on1 = ['Report Number', 'Local Case Number' ,'Agency Name', 'ACRS Report Type' ,'Crash Date/Time',
            'Route Type' ,'Road Name', 'Cross-Street Name' ,'Off-Road Description' ,'Municipality',
            'Related Non-Motorist' ,'Collision Type','Weather','Surface Condition','Light',
            'Traffic Control','Driver Substance Abuse','Non-Motorist Substance Abuse',
            'Latitude','Longitude','Location']

merge_on2 = [ 'Report Number', 'Local Case Number' ,'Agency Name', 'ACRS Report Type' ,'Crash Date/Time',
            'Route Type' ,'Road Name', 'Cross-Street Name' ,'Off-Road Description' ,'Municipality',
            'Related Non-Motorist' ,'Collision Type','Weather','Surface Condition','Light',
            'Traffic Control','Driver Substance Abuse','Non-Motorist Substance Abuse',
            'Latitude','Longitude','Location','Person ID','At Fault','Injury Severity']

merger1 = driver_data.merge(incidents_data, on= merge_on1, how='outer')
crash_merged = merger1.merge(nonmotor_data, on = merge_on2, how = 'outer')

crash_merged['Crash Date/Time'] = pd.to_datetime(crash_merged['Crash Date/Time'])

In [ ]:
#Time of day
def categorize_time_of_day(dt):
  hour = dt.hour
  if 3 <= hour < 7:
      return "Early Morning"
  elif 8 <= hour < 12:
       return "Morning"
  elif 12 <= hour < 17:
       return "Afternoon"
  elif 17 <= hour < 21:
       return "Evening"
  else:
       return "Late Night"

In [ ]:
#Renaming location
crash_merged = crash_merged.rename(columns= {'Location':'Geolocation'})

#Making New features
#Date and Time
crash_merged['Year'] = crash_merged['Crash Date/Time'].dt.year
crash_merged['Month'] = crash_merged['Crash Date/Time'].dt.month_name()
crash_merged['Time'] = crash_merged['Crash Date/Time'].dt.time

#Holiday- MD
us_holidays = holidays.US(state='MD')
crash_merged['Holiday'] = crash_merged['Crash Date/Time'].apply(lambda x: us_holidays.get(x, "Not a Holiday"))

#Day Type
crash_merged['Day Type'] = crash_merged['Crash Date/Time'].apply(lambda x: 'Weekend' if x.weekday() >= 5 else 'Weekday')

#Day of Week
crash_merged['Day of Week'] = crash_merged['Crash Date/Time'].dt.day_name()

#Time of Day
crash_merged['Time of Day'] = crash_merged['Crash Date/Time'].apply(categorize_time_of_day)

BELOW WE ARE REPLACING VALUES FOR CONSISTENCY

In [ ]:
#Crash Replacement for Standardization - Agency Name
#replacements
rockville = 'Rockville Police Department'
takoma = 'Takoma Park Police Department'
gaithersburg = ' Gaithersburg Police Department'
mcpark = 'Maryland-National Capital'
montgomery = 'Montgomery County Police'

#REplacing
replacements_Agency = {"Rockville Police Departme": rockville,
                "ROCKVILLE": rockville,
                'TAKOMA': takoma,
                'Takoma Park Police Depart': takoma,
                'Gaithersburg Police Depar':gaithersburg,
                'GAITHERSBURG':gaithersburg,
                'MCPARK':mcpark,
                'MONTGOMERY':montgomery,
               }

crash_merged['Agency Name'] = crash_merged['Agency Name'].replace(replacements_Agency)

In [ ]:
#Crash Replacement for Standardization - Route Type
#replacements
md_state = 'Maryland (State) Route'
municip_route = 'Municipality Route'
county_route = 'County Route'
gov_route = 'Government Route'


#REplacing
replacements_Route = {'Maryland (State)': md_state,
                'Municipality': municip_route,
                'County': county_route,
                'Government': gov_route,

               }

crash_merged['Route Type'] = crash_merged['Route Type'].replace(replacements_Route)

In [ ]:
#Crash Replacement for Standardization - related non-motorist
#replacements
pedestrian ='Pedestrian'
cyclist = 'Cyclist'
wheelchair = 'Wheelchair'
scooter = 'Scooter'
machine_ops = 'Machine Operator'
not_in_transport = 'Stationary Vehicle Occupant'

animal = 'Animal-Drawn Vehicle'
unknown = 'Unknown'
other = 'Other'

#REplacing
replacements_non_motorist = {
                'PEDESTRIAN' : pedestrian,
                'OTHER, PEDESTRIAN' : pedestrian,
                'OTHER CONVEYANCE, PEDESTRIAN' : pedestrian,
                'Other Pedestrian (person in a building, skater, personal conveyance, etc.), Pedestrian' : pedestrian,
                'Other Pedestrian (person in a building, skater, personal conveyance, etc.)' : pedestrian,

                'BICYCLIST' : cyclist,
                'Cyclist (non-electric)' :cyclist,
                'OTHER PEDALCYCLIST' :cyclist,
                'OTHER PEDALCYCLIST' :cyclist,
                'BICYCLIST, PEDESTRIAN' :cyclist,
                'BICYCLIST, OTHER' :cyclist,
                'Cyclist (Electric)' :cyclist,

                'Wheelchair (electric)' :wheelchair,
                'Unknown, Wheelchair (electric)' :wheelchair,
                'Wheelchair (non-electric)' :wheelchair,

                'Scooter (electric)' :scooter,
                'Scooter (non-Electric)' :scooter,

                'OTHER' :other,
                'OTHER, OTHER CONVEYANCE' :other,
                'OTHER CONVEYANCE' :other,

                'MACHINE OPERATOR/RIDER' :machine_ops,
                'MACHINE OPERATOR/RIDER, OTHER' :machine_ops,

                'Occupant of Motor Vehicle Not in Transport' :not_in_transport,
                'Occupant of Motor Vehicle Not in Transport Pedestrian' : not_in_transport ,
                'Occupant of Motor Vehicle Not in Transport, Pedestrian' : not_in_transport ,

                'Unknown' : unknown,
                'Unknown Type Of Non-Motorist' : unknown,

                'IN ANIMAL-DRAWN VEH' : animal
                }




crash_merged['Related Non-Motorist'] = crash_merged['Related Non-Motorist'].replace(replacements_non_motorist)

In [ ]:
#Crash Replacement for Standardization - Collision Type
#replacements
head_on = 'Head-on'
headon_left = 'Head-on, Left Turn'
rear_end = 'Rear End'
same_swipe = 'Same Direction, Side Swipe'
oppsite_swipe = 'Opposite Direction, Side Swipe'
angle_col ='Angle Collision'

other_unknown = 'Other/Unknown'

#REplacing




                  #Head on
replacements_Collision = { 'HEAD ON': head_on,
                  'Front to Front': head_on,

                  #Head on but with angle
                  'ANGLE MEETS LEFT HEAD ON':headon_left,
                  'HEAD ON LEFT TURN':headon_left,


                  #Left turns
                  'SAME DIR BOTH LEFT TURN': 'Same Direction Left Turn, Both',

                  'SAME DIRECTION LEFT TURN': 'Same Direction Left Turn',

                  'OPPOSITE DIR BOTH LEFT TURN': 'Oppsite Direction Left Turn, Both',

                  #Right turn
                  'SAME DIRECTION RIGHT TURN': 'Same Direction Right Turn',



                  #Rear ends
                  'SAME DIR REAR END':rear_end,
                  'Front to Rear':rear_end,
                  #rear ends while turning left
                  'SAME DIR REND RIGHT TURN': 'Same Direction Rear End, Right Turn',
                  #rear ends while turning right
                  'SAME DIR REND LEFT TURN':'Same Direction Rear End, Left Turn',

                  #these can stay
                  #'Rear To Rear':,
                  #'Rear To Side':,

                  #Sideswipe
                  'SAME DIRECTION SIDESWIPE':same_swipe,
                  'Sideswipe, Same Direction':same_swipe,
                  #Sideswipe - oppsite direction.
                  'OPPOSITE DIRECTION SIDESWIPE':oppsite_swipe,
                  'Sideswipe, Opposite Direction':oppsite_swipe,

                  #Angle Collisons
                  'STRAIGHT MOVEMENT ANGLE':angle_col, #Figure out what this means
                  'Angle': angle_col,


                  'ANGLE MEETS RIGHT TURN': 'Right-Turn Angle Impact', # A vehicle making a right turn is hit at an angle by another vehicle.
                  'ANGLE MEETS LEFT TURN': 'Left-Turn Angle Impact',



                  #Single Vehicale
                  'SINGLE VEHICLE': 'Single Vehicle',


                  # Other and Unknowns
                  'UNKNOWN':other_unknown,
                  'Unknown':other_unknown,

                  'OTHER':other_unknown,
                  'Other':other_unknown ,
               }

crash_merged['Collision Type'] = crash_merged['Collision Type'].replace(replacements_Collision)

In [ ]:
#Crash Replacement for Standardization - Weather
#replacements
foggy = 'Foggy'


#Replacing
replacements_Weather = { 'CLEAR': 'Clear',
                  'CLOUDY': 'Cloudy',
                  'RAINING':'Rain',
                  'SNOW':'Snow',
                  'WINTRY MIX':'Wintry Mix',
                  'BLOWING SAND, SOIL, DIRT': 'Blowing Sand, Soil, Dirt',
                  'BLOWING SNOW':'Blowing Snow',

                  'FOGGY':foggy,
                  'Fog, Smog, Smoke':foggy,

                  'SEVERE WINDS': 'Severe Crosswinds',
                  'SLEET':'Sleet Or Hail',
                  'Freezing Rain Or Freezing Drizzle': 'Freezing Rain',

                  'UNKNOWN': 'Unknown',
                  'OTHER': 'Other',


               }

crash_merged['Weather'] = crash_merged['Weather'].replace(replacements_Weather)

In [ ]:
#Crash Replacement for Standardization - Surface Condition
#replacements

#Replacing
replacements_surface = {
    'DRY': 'Dry',
    'WET': 'Wet',
    'SNOW': 'Snow',
    'ICE': 'Ice/Frost',
    'OTHER': 'Other',
    'SLUSH':  'Slush',
    'MUD, DIRT, GRAVEL':  'Mud, Dirt, Gravel',
    'OIL': 'Oil',
    'WATER(STANDING/MOVING)':  'Water(standing, moving)',
    'SAND': 'Sand'

}

crash_merged['Surface Condition'] = crash_merged['Surface Condition'].replace(replacements_surface)

In [ ]:
#Crash Replacement for Standardization -Light
#replacements



#Replacing
replacements_Light = {
    'DAYLIGHT': 'Daylight',
    'DUSK': 'Dusk',
    'DARK LIGHTS ON': 'Dark- Lighted',
    'DAWN' : 'Dawn',
    'DARK NO LIGHTS':'Dark- Not Lighted',
    'UNKNOWN': 'Unknown',
    'OTHER': 'Other',
    'DARK -- UNKNOWN LIGHTING': 'Dark- Unknown Lighting',

 }

crash_merged['Light'] = crash_merged['Light'].replace(replacements_Light)

In [ ]:
#Crash Replacement for Standardization - Traffic Controls
#Standard replacement value- keys
traffic_signal = 'Traffic Signal' #d
yield_sign = 'Yield Sign' #d
no_control = 'No Controls'#d
stop_sign = 'Stop Sign'
person_control = 'Person'
warning_sign = ' Warning Sign'
flashing_signal = 'Flashing Traffic Signal'
ped_crossing_sign = 'Crossing Sign- Pedestrian'
ped_crossing_signal = 'Pedestrian- Traiffic Signal'
ped_crossing_stop_sign = 'Pedestrian- Crossing with Stop Sign'
bike_cross = 'Crossing Sign- Bicycle'
lane_use = 'Lane Use Control Signal'
school_zone = 'School Zone Signs'
rail_road = 'Crossing Signal- Flashing Railroad'
curve_ahead = 'Warning Sign- Curve Ahead'
intersection_ahead = 'Warning Sign- Intersection Ahead'
other_signal = 'Other Signal'



#Variants
traffic_signal_variants = {
    'TRAFFIC SIGNAL',
    'Traffic Control Signal',
    'Other, Traffic Control Signal',
    'Other Signal, Traffic Control Signal',
    'Other Warning Sign, Traffic Control Signal',
    'Flashing Traffic Control Signal',

}

yield_sign_variants = {
    'YIELD SIGN',
    'Yield Sign',
    'No Controls, Yield Sign',
    'Traffic Control Signal, Yield Sign',
}

no_control_variants = {
    'NO CONTROLS',
    'No Controls',
    'No Controls, Other',

}
'''
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
EDIT_variants = { }
'''



replacements = {
    **{key: traffic_signal for key in traffic_signal_variants},
    **{key: yield_sign for key in yield_sign_variants},
    **{key: no_control for key in no_control_variants}
}
# Replace values in the column
#crash_merged['Traffic Control'] = crash_merged['Traffic Control'].replace(
#    {key: traffic_signal for key in traffic_signal_variants})

#crash_merged['Traffic Control'] = crash_merged['Traffic Control'].replace(
#    {key: yield_sign for key in yield_sign_variants})

#crash_merged['Traffic Control'] = crash_merged['Traffic Control'].replace(
#    {key: no_control for key in no_control_variants})




#crash_merged['Traffic Control'] = crash_merged['Traffic Control'].replace(
    #{key: EDIT for key in EDIT_variants})


crash_merged['Traffic Control'] = crash_merged['Traffic Control'].replace(replacements)

In [ ]:
#Crash Replacement for Standardization - Driver Substance Abuse
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Non-Motorist Substance Abuse
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Injury Severity
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization -Circumstance
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Driver Distracted By
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Vehicle Damage Extent
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Vehicle Body Type
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Vehicle Movement
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Lane Type
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Road Grade
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - First Harmful Event
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Second Harmful Event
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Junction
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Intersection Type
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Road Alignment
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Road Condition
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Road Division
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Pedestrian Type
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Pedestrian Movement
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Pedestrian Actions
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Pedestrian Location
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization - Safety Equipment
#replacements



#Replacing
replacements_ = {  }

crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
#Crash Replacement for Standardization -
#replacements



#Replacing
#replacements_ = {  }

#crash_merged['c'] = crash_merged['c'].replace(replacements_)

In [ ]:
# Looking int chosen datatset
look_in = crash_merged

look_in.info()
print('~~~~~~~~~~~~~~~~~~~~~')
look_in.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224247 entries, 0 to 224246
Data columns (total 67 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Report Number                  224247 non-null  object        
 1   Local Case Number              224247 non-null  object        
 2   Agency Name                    224247 non-null  object        
 3   ACRS Report Type               224247 non-null  object        
 4   Crash Date/Time                224247 non-null  datetime64[ns]
 5   Route Type                     200899 non-null  object        
 6   Road Name                      198259 non-null  object        
 7   Cross-Street Name              186263 non-null  object        
 8   Off-Road Description           22138 non-null   object        
 9   Municipality                   21446 non-null   object        
 10  Related Non-Motorist           13309 non-null   object        
 11  

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Off-Road Description,Municipality,...,Pedestrian Actions,Pedestrian Location,Safety Equipment,Year,Month,Time,Holiday,Day Type,Day of Week,Time of Day
0,DD5502000M,15004827,Rockville Police Departme,Property Damage Crash,2015-01-30 12:16:00,Maryland (State),W JEFFERSON ST,S WASHINGTON ST,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,January,12:16:00,Not a Holiday,Weekday,Friday,Afternoon
1,DD5502000M,15004827,Rockville Police Departme,Property Damage Crash,2015-01-30 12:16:00,Maryland (State),W JEFFERSON ST,S WASHINGTON ST,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,January,12:16:00,Not a Holiday,Weekday,Friday,Afternoon
2,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon
3,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon
4,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon


In [ ]:
#Checking what is null and uniques vaules in colume

#print('Here are the number of Null in each feature\n',look_in.isnull().sum())

#Look into which colume
col ='Holiday'

print(f'\n Number of Unique values in {col}: ',(look_in[col].nunique()), '\nThis Colume has:', set(look_in[col]))


 Number of Unique values in Holiday:  20 
This Colume has: {"Presidents' Day", 'Veterans Day', 'Christmas Day (observed)', 'Inauguration Day', 'Veterans Day (observed)', 'Christmas Day', 'Thanksgiving', "New Year's Day (observed)", 'Juneteenth National Independence Day (observed)', 'Independence Day (observed)', "New Year's Day", 'Martin Luther King Jr. Day', 'Labor Day', 'Juneteenth National Independence Day', 'Memorial Day', 'Columbus Day', 'Not a Holiday', 'Inauguration Day; Martin Luther King Jr. Day', 'Independence Day', 'American Indian Heritage Day'}


In [ ]:
# Check all features and unique values
unique__whole = {}

exclude_columns = ['Report Number', 'Local Case Number','Crash Date/Time', 'Time',
                   'Latitude','Longitude','Geolocation', 'Distance',
                   'Person ID', 'Vehicle ID', 'Off-Road Description',
                   'Vehicle Model','Vehicle Make','Vehicle Year', 'Lane Direction',
                   'Vehicle First Impact Location','Road Name','Cross-Street Name']

# loop through the columns
for col in crash_merged.columns:
  if col not in exclude_columns:
    unique__whole[col] = {
      'unique_count': crash_merged[col].nunique(),
      'unique_values': crash_merged[col].unique().tolist()
  }

# Convert to dataframe
unique__whole_df = pd.DataFrame.from_dict(unique__whole, orient='index')

In [ ]:
# Save it as a csv
unique__whole_df.to_csv('unique__whole_df.csv')
print("Dictionary saved to unique_values_info.csv successfully!")
print('Features', len(crash_merged.columns), '\nSamples',len(crash_merged))

Dictionary saved to unique_values_info.csv successfully!
Features 67 
Samples 224247


In [ ]:
#Save new CSV
crash_merged.to_csv('crash_merged.csv',)

#VIz

In [ ]:
#Histogram
crash_df =pd.DataFrame(crash_merged)
crash_df['Year'].hist( edgecolor = '#f2f2f2', bins=10)


plt.xticks(rotation = 90)
plt.grid( axis= 'x')


plt.show()

In [ ]:
#Barchart

yearly_count = crash_merged['Year'].value_counts().sort_index()


# Normalize values for color mapping
norm = (yearly_count - yearly_count.min()) / (yearly_count.max() - yearly_count.min())  # Normalize between 0 and 1
colors = plt.cm.RdYlGn_r(norm)


plt.figure(figsize=(10, 6))
plt.bar(yearly_count.index, yearly_count.values, align='center', color= colors)



plt.xlabel('Year')
plt.ylabel('TotalNumber of Incidents')
plt.title('Number of Incidents Through the Years')
plt.xticks(yearly_count.index, rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()